In [1]:
import pandas as pd
import numpy as np

#Import dataset
data=pd.read_excel("Dataset_Marketplace.xlsx")
data.head()

,Tweet,Object,Sentiment
0,b'@BukaBantuan tolong cepat bantu proses order...,Bukalapak,Negatif
1,b'@BukaBantuan Kalau bisa sih barang nya di ja...,Bukalapak,Negatif
2,"b'@BukaBantuan Terima kasih, kak. Kendala suda...",Bukalapak,Positif
3,b'@Ndons_Back: Min @bukalapak @BukaBantuan pej...,Bukalapak,Negatif
4,b'@BukaBantuan Belum ada masuk kak email nya',Bukalapak,Negatif


In [2]:
#Cleansing
import string 
import re #regex library

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").
        replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#x][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).
        split())
    # remove b',RT, and &amp
    text = text.replace("b'","").replace('amp',"").replace('RT',"")
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['Tweet'] = data['Tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['Tweet'] = data['Tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['Tweet'] = data['Tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['Tweet'] = data['Tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['Tweet'] = data['Tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['Tweet'] = data['Tweet'].apply(remove_singl_char)

#view
data['tweet_cleansing'] = data['Tweet'].apply(remove_singl_char)

print('Cleansing Result : \n') 
print(data['Tweet'].head())
print('\n\n\n')

Cleansing Result : 

0    tolong cepat bantu proses orderan diteruskan k...
1    Kalau bisa sih barang nya di jadiin aja jangan...
2    Terima kasih kak Kendala sudah terselesaikan s...
3    Back Min pejualan film porno banyak di tempat ...
4                        Belum ada masuk kak email nya
Name: Tweet, dtype: object






In [3]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data['tweet_casefolding'] = data['tweet_cleansing'].str.lower()


print('Case Folding Result : \n')
print(data['tweet_casefolding'].head())
print('\n\n\n')

Case Folding Result : 

0    tolong cepat bantu proses orderan diteruskan k...
1    kalau bisa sih barang nya di jadiin aja jangan...
2    terima kasih kak kendala sudah terselesaikan s...
3    back min pejualan film porno banyak di tempat ...
4                        belum ada masuk kak email nya
Name: tweet_casefolding, dtype: object






In [4]:
def convert_negation(text):
    text=''.join(i+' ' if i != 'gak' else i for i in text.split())
    text=''.join(i+' ' if i != 'ga' else i for i in text.split())
    text=''.join(i+' ' if i != 'ngga' else i for i in text.split())
    text=''.join(i+' ' if i != 'gk' else i for i in text.split())
    text=''.join(i+' ' if i != 'nggak' else i for i in text.split())
    text=''.join(i+' ' if i != 'enggak' else i for i in text.split())
    text=''.join(i+' ' if i != 'tidak' else i for i in text.split())
    text=''.join(i+' ' if i != 'tdk' else i for i in text.split())
    text=''.join(i+' ' if i != 'tida' else i for i in text.split())
    text=''.join(i+' ' if i != 'tak' else i for i in text.split())
    text=''.join(i+' ' if i != 'bukan' else i for i in text.split())
    text=''.join(i+' ' if i != 'bkan' else i for i in text.split())
    text=''.join(i+' ' if i != 'bkn' else i for i in text.split())
    text=''.join(i+' ' if i != 'jangan' else i for i in text.split())
    text=''.join(i+' ' if i != 'jgn' else i for i in text.split())
    return ''.join(i+' ' if i != 'kurang' else i for i in text.split())
              
data['tweet_convertnegation'] = data['tweet_casefolding'].apply(convert_negation)
data['tweet_convertnegation'].head(80)

0     tolong cepat bantu proses orderan diteruskan k...
1     kalau bisa sih barang nya di jadiin aja jangan...
2     terima kasih kak kendala sudah terselesaikan s...
3     back min pejualan film porno banyak di tempat ...
4                        belum ada masuk kak email nya 
                            ...                        
75                     iya dong min biar bisa on terus 
76                 makasih min sukses selalu bukalapak 
77                                   ini knp gkbisa ya 
78    halooo barang sudah dipick up driver tapi enta...
79    karena sepertinya mulai ketahuan saya arahkan ...
Name: tweet_convertnegation, Length: 80, dtype: object

In [5]:
# ------ Tokenizing ---------
# import word_tokenize from NLTK
from nltk.tokenize import word_tokenize 

def tokenizing(text):
    return word_tokenize(text)

data['tweet_tokens'] = data['tweet_convertnegation'].apply(tokenizing)

print(data['tweet_tokens'].head(80))
print('\n\n\n')

0     [tolong, cepat, bantu, proses, orderan, diteru...
1     [kalau, bisa, sih, barang, nya, di, jadiin, aj...
2     [terima, kasih, kak, kendala, sudah, terselesa...
3     [back, min, pejualan, film, porno, banyak, di,...
4                  [belum, ada, masuk, kak, email, nya]
                            ...                        
75              [iya, dong, min, biar, bisa, on, terus]
76            [makasih, min, sukses, selalu, bukalapak]
77                               [ini, knp, gkbisa, ya]
78    [halooo, barang, sudah, dipick, up, driver, ta...
79    [karena, sepertinya, mulai, ketahuan, saya, ar...
Name: tweet_tokens, Length: 80, dtype: object






In [6]:
normalization_word = pd.read_excel("normalization.xlsx")

normalization_word_dict = {}

for index, row in normalization_word.iterrows():
    if row[0] not in normalization_word_dict:
        normalization_word_dict[row[0]] = row[1] 

def normalization_term(document):
    return [normalization_word_dict[term] if term in normalization_word_dict else term for term in document]

data['tweet_normalization'] = data['tweet_tokens'].apply(normalization_term)

data['tweet_normalization'].head(80)

0     [tolong, cepat, bantu, proses, orderan, diteru...
1     [kalau, bisa, sih, barang, nya, di, jadikan, s...
2     [terima, kasih, kakak, kendala, sudah, tersele...
3     [kembali, admin, pejualan, film, pornografi, b...
4                [belum, ada, masuk, kakak, email, nya]
                            ...                        
75            [iya, dong, admin, biar, bisa, on, terus]
76      [terimakasih, admin, sukses, selalu, bukalapak]
77                        [ini, kenapa, tidakbisa, iya]
78    [halo, barang, sudah, dijemput, up, sopir, tap...
79    [karena, sepertinya, mulai, ketahuan, saya, ar...
Name: tweet_normalization, Length: 80, dtype: object

In [7]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stopword_factory = StopWordRemoverFactory()
stopwords_txt = open("stopword.txt", "r")


stopword_dict = []
for word in stopwords_txt:
    stopword_dict.append(word.rstrip())
list_stopword=stopword_factory.get_stop_words()+stopword_dict

def remove_stopword(words):
    return [word for word in words if word not in list_stopword]

data['tweet_stopwordremoval']= data['tweet_normalization'].apply(remove_stopword)
data['tweet_stopwordremoval'].head(80)

0     [cepat, bantu, proses, orderan, diteruskan, bu...
1                         [barang, kembalikan, dananya]
2               [terima, kasih, kendala, terselesaikan]
3     [admin, pejualan, film, pornografi, janganrusa...
4                                        [masuk, email]
                            ...                        
75                                          [admin, on]
76              [terimakasih, admin, sukses, bukalapak]
77                                          [tidakbisa]
78    [barang, dijemput, sopir, barangnya, status, p...
79                       [arahkan, transfer, dana, sms]
Name: tweet_stopwordremoval, Length: 80, dtype: object

In [8]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def Stem(remove_stopword): 
    return [stemmer.stem(word) for word in remove_stopword]

data['tweet_stemming']= data['tweet_stopwordremoval'].apply(Stem)
data['tweet_stemming'].head(80)

0     [cepat, bantu, proses, order, terus, bukadompe...
1                               [barang, kembali, dana]
2                     [terima, kasih, kendala, selesai]
3     [admin, jual, film, pornografi, janganrusak, p...
4                                        [masuk, email]
                            ...                        
75                                          [admin, on]
76              [terimakasih, admin, sukses, bukalapak]
77                                          [tidakbisa]
78    [barang, jemput, sopir, barang, status, jual, ...
79                          [arah, transfer, dana, sms]
Name: tweet_stemming, Length: 80, dtype: object

In [9]:
data.to_excel("Preprocessing.xlsx")